## Generating a random set partition of a given size $n$.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.special
from mpmath import *
import random
import time

In [2]:
n=1_000_000

In [3]:
def find_the_mode (n): #Inefficient algorithm but it is not central
    for m in range(1,n+1):
        if math.log(1+1/m)*n <= math.log(1+m):
            return(m)
    raise Exception('Problème calcul du mode')

m = find_the_mode(n)

print(m)
m = m

87847


The precision doubling procedure is not yet implemented

In [4]:
mp.dps = 10*(math.log(n) + 10)
def W (n):
    beta = mp.ln(n) -mp.ln(mp.ln(n))
    for i in range (15): # je reste à 15 pour l'instant : ça fait 5 000 décimales de précision donc c'est pas ça qui coince en théorie
        beta = beta*(1+mp.ln(n/beta))/(1+beta)
    return(beta)

def compute_B_n_tilde (n):
    return((3/(2*mp.sqrt(n)))*(mp.power(n/W(n),n+1/2))*mp.exp(n/W(n)-n-1))

def g_int (m,n):
    if m < 0 : return(0)
    return(mp.power(m,n)/mp.factorial(m))

def g_function (x,m,n):
    g_m = g_int (m,n)
    B_n_tilde = compute_B_n_tilde(n)
    facteur = mp.exp(1-g_m*(mp.absmax(x-m)-mp.mpf('0.5'))/(mp.exp(1)*B_n_tilde))
    if facteur < 1 :
        return(g_m*facteur)
    else:
        return(g_m)

In [5]:
def tirer_Y (n,m):
    g_m = g_int (m,n)
    B_n_tilde = compute_B_n_tilde(n)
    bern = random.random()
    if bern < 2/(g_m/(mp.exp(1)*B_n_tilde)+4) : #on est dans l'un des deux cotés
        dg = random.randint(1,2)
        expdistribution = mp.mpf('-1')*mp.log(random.random())*mp.exp(1)*B_n_tilde/g_m
        if dg == 1 : return(m + 0.5 + mp.exp(1)*B_n_tilde/g_m + expdistribution)
        else : return(m -0.5 - mp.exp(1)*B_n_tilde/g_m - expdistribution)
    else:
        return(m + (1 - 2*random.random())*(mp.mpf('0.5') + mp.exp(1)*B_n_tilde/g_m))

In [6]:
def tirer_k (n,m): #Generates $U$
    b = True
    while b :
        Y = tirer_Y(n,m)
        X = round(Y)
        if random.random()*g_function(Y,m,n) < g_int(X,n):
            b = False
    return(X)

If one wants to look at the distribution of $Y$ and $U$.

In [7]:
# results = [tirer_k(n,m) for _ in range(1000)]
# results2 = [round(tirer_Y(n,m)) for _ in range(1000)]
# # Calculate the mean and standard deviation
# mean = np.mean(results)
# std_dev = np.std(results)

# print(f"Mean of outcomes: {mean}")
# print(f"Standard deviation of outcomes: {std_dev}")


# # Plot the histogram
# plt.hist(results, bins=30, density=True, alpha=0.6, color='g', label='Outcomes')
# plt.hist(results2, bins=30, density=True, alpha=0.6, color='b', label='Random abscissa weighted with g(x)')

# # Overlay the expected Gaussian distribution
# x = np.linspace(min(results), max(results), 1000)
# gaussian = (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)
# plt.plot(x, gaussian, 'r-', label='Gaussian Fit')

# # Add labels and legend
# plt.xlabel('Outcome')
# plt.ylabel('Density')
# plt.title('Distribution of tirer_W1() Outcomes')
# plt.legend()

# # Show the plot
# plt.show()

In [8]:
def remplissage_urnes (k,n): 
    urnes = []
    # t = 0
    for i in range (k):
        urnes.append([])
    for i in range (n):
        # gtavant = time.time()
        u = random.randint(0,k-1)
        # gtapres = time.time()
        # t+= (gtapres - gtavant)
        urnes[u].append(i)
    partition = []
    for i in range (k):
        if urnes[i] != []:
            partition.append(urnes[i])
    # print("temps passé à génerer des nombres" ,t)
    return(partition)

Remark: for big $n$ (say $n\sim 10^8$) it is much quicker to restart the kernel before calling the program.

In [10]:
debut = time.time()

m = find_the_mode(n)
k = tirer_k(n,m)

print("There are",k,"urns")

apres_k = time.time()

print("Time to generate U : " , apres_k - debut)

partition = remplissage_urnes(k,n)
print("First equivalence class : ",partition[0])
print("Number of subsets : ",len(partition))
print("Expectancy of the number of subsets (rounded) : ",round(n/W(n)))
print("Difference to the expectancy in the number of subsets : ",round(n/W(n)-len(partition)))
print("Standard deviation (rounded) = ", round(np.sqrt(n)/W(n)))
fin = time.time()


print("Total time of the procedure : ",fin - apres_k)

There are 87889 urns
Time to generate U :  0.09140419960021973


First equivalence class :  [134883, 151796, 296707, 365522, 386580, 488104, 500330, 532841, 545703, 563007, 580196, 585588, 758724, 849748]
Number of subsets :  87889
Expectancy of the number of subsets (rounded) :  87848
Difference to the expectancy in the number of subsets :  -41
Standard deviation (rounded) =  88
Total time of the procedure :  1.7974483966827393
